In [28]:
import numpy as np 
import pandas as pd 
import geopandas as gpd 
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import contextily as cx 
import plotly.express as px 
import plotly.io as pio
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()
import rasterio 
from rasterio import plot as rioplot 
from rasterio.mask import mask  
from linearmodels import PooledOLS 
from linearmodels import PanelOLS 
from linearmodels import BetweenOLS 
from linearmodels.panel.results import compare 
import inequality 
from shapely.geometry import mapping
import os
import glob

# Set the float format to prevent scientific notation
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)

In [39]:
# Specify the path to your zip file
# zip_file_path = "/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/INDIAN-SHAPEFILES-master/sub_dist.zip"
# zip_file_path = "/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/INDIAN-SHAPEFILES-master/Dist_2022.zip"

# Read the shapefile from the zip file using GeoPandas
gdf = gpd.read_file('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/maps-master/geojson/sub_dist.geojson')

# Now you can work with the GeoDataFrame (gdf) containing the shapefile data
gdf_NTL = gdf.copy()
gdf

,NAME,DISTRICT,STATE_UT,geometry
0,DATA NOT AVAILABLE,DATA NOT AVAILABLE,Jammu & Kashmir,"MULTIPOLYGON (((74.77117 37.01042, 74.77968 36..."
1,NaN,Kachchh,Gujarat,"MULTIPOLYGON (((70.12473 22.97347, 70.12521 22..."
2,KACHCHH (PART),Kachchh,Gujarat,"MULTIPOLYGON (((71.30731 24.56554, 71.30731 24..."
3,Kupwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.57425 34.59931, 74.56734 34..."
4,Handwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.13457 34.44413, 74.13615 34..."
...,...,...,...,...
6070,Mayabunder,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.72334 12.73736, 92.72345 12..."
6071,Rangat,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.75932 12.16706, 92.75909 12..."
6072,Ferrargunj,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.60467 11.53490, 92.60471 11..."
6073,Port Blair,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.61728 10.92835, 92.61446 10..."


In [3]:
def replace_negative(radiance_datacube, replacement=np.nan):
    # Create a mask for negative values
    negative_mask = radiance_datacube < 0
    
    # Replace negative values with replacement
    radiance_datacube[negative_mask] = replacement
    
    return radiance_datacube

In [8]:
startYear = 2001
endYear = 2012

In [24]:

for year in range ( startYear , endYear + 1 ) :
    # The raster file of the given year is loaded
    rasterFile = rasterio.open("/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/VIIRS_Monthly_Tiled/2020/20"+str(year)+".tif")
    # A mask is a created using the geometry provided in the vector file
    def clean_mask ( geom , dataset = rasterFile , **mask_kw ):
        mask_kw.setdefault("crop" , True )
        mask_kw.setdefault("all_touched" , True )
        
        masked , mask_transform = mask (dataset = dataset , shapes = (geom,) ,**mask_kw )
        return masked

statewise_agg_ntl = gdf.geometry.apply(clean_mask).apply(np.ma.sum)

gdf_NTL[str(year)] = statewise_agg_ntl

In [23]:
gdf_NTL

,NAME,DISTRICT,STATE_UT,geometry,2012
0,DATA NOT AVAILABLE,DATA NOT AVAILABLE,Jammu & Kashmir,"MULTIPOLYGON (((74.77117 37.01042, 74.77968 36...",124124.63281
1,NaN,Kachchh,Gujarat,"MULTIPOLYGON (((70.12473 22.97347, 70.12521 22...",9427.25977
2,KACHCHH (PART),Kachchh,Gujarat,"MULTIPOLYGON (((71.30731 24.56554, 71.30731 24...",9764.55957
3,Kupwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.57425 34.59931, 74.56734 34...",4129.37988
4,Handwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.13457 34.44413, 74.13615 34...",1798.93994
...,...,...,...,...,...
6070,Mayabunder,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.72334 12.73736, 92.72345 12...",-57.33000
6071,Rangat,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.75932 12.16706, 92.75909 12...",-123.59998
6072,Ferrargunj,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.60467 11.53490, 92.60471 11...",306.91000
6073,Port Blair,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.61728 10.92835, 92.61446 10...",2352.09009


In [25]:
gdf_NTL

,NAME,DISTRICT,STATE_UT,geometry,2012
0,DATA NOT AVAILABLE,DATA NOT AVAILABLE,Jammu & Kashmir,"MULTIPOLYGON (((74.77117 37.01042, 74.77968 36...",212641.87500
1,NaN,Kachchh,Gujarat,"MULTIPOLYGON (((70.12473 22.97347, 70.12521 22...",9146.98047
2,KACHCHH (PART),Kachchh,Gujarat,"MULTIPOLYGON (((71.30731 24.56554, 71.30731 24...",30467.77539
3,Kupwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.57425 34.59931, 74.56734 34...",4566.95020
4,Handwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.13457 34.44413, 74.13615 34...",2588.09009
...,...,...,...,...,...
6070,Mayabunder,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.72334 12.73736, 92.72345 12...",669.07996
6071,Rangat,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.75932 12.16706, 92.75909 12...",1251.92993
6072,Ferrargunj,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.60467 11.53490, 92.60471 11...",1357.66003
6073,Port Blair,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.61728 10.92835, 92.61446 10...",2502.69019


In [26]:
gdf_NTL = gdf_NTL.drop('geometry',axis=1)

In [27]:
gdf_NTL.to_csv('/Users/sid/Desktop/test.csv',index=False)

In [40]:
# Loop through each month
for month in range(1, 13):
    # Construct the path to the raster file for the current month
    raster_path = f"/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/VIIRS_Monthly_Tiled/2020/2020{month:02d}.tif"
    
    # Open the raster file
    with rasterio.open(raster_path) as rasterFile:
        # Define a function to create mask for the current month
        def clean_mask(geom, dataset=rasterFile, **mask_kw):
            mask_kw.setdefault("crop", True)
            mask_kw.setdefault("all_touched", True)
            masked, mask_transform = mask(dataset=dataset, shapes=[geom], **mask_kw)
            return masked
        
        # Apply mask to each geometry and calculate sum of luminosity for the current month
        statewise_agg_ntl = gdf.geometry.apply(clean_mask).apply(np.ma.sum)
        
        # Add a column to the DataFrame for the current month
        gdf_NTL[f"2020{month:02d}"] = statewise_agg_ntl

gdf_NTL

,NAME,DISTRICT,STATE_UT,geometry,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012
0,DATA NOT AVAILABLE,DATA NOT AVAILABLE,Jammu & Kashmir,"MULTIPOLYGON (((74.77117 37.01042, 74.77968 36...",124124.63281,193721.37500,162904.81250,179708.37500,177151.42188,32355.73242,159437.59375,155195.57812,167450.29688,198710.98438,241777.53125,212641.87500
1,NaN,Kachchh,Gujarat,"MULTIPOLYGON (((70.12473 22.97347, 70.12521 22...",9427.25977,8750.15039,8546.46094,7721.16992,7799.17969,9047.06055,7609.64941,4020.55005,7146.41992,8381.15039,8549.86914,9146.98047
2,KACHCHH (PART),Kachchh,Gujarat,"MULTIPOLYGON (((71.30731 24.56554, 71.30731 24...",9764.55957,32678.37695,40663.44531,48432.02734,41855.58203,41424.16406,33486.19922,28309.69336,36691.76562,35497.92578,38389.41016,30467.77539
3,Kupwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.57425 34.59931, 74.56734 34...",4129.37988,4396.91992,3605.95996,4178.94971,4703.93994,0.00000,4575.87012,4767.25000,5000.12988,4962.14990,5137.04004,4566.95020
4,Handwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.13457 34.44413, 74.13615 34...",1798.93994,2190.91016,1783.22986,2253.55005,2381.04004,481.53998,2350.43994,2451.75000,2431.86011,2449.30005,2696.09009,2588.09009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,Mayabunder,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.72334 12.73736, 92.72345 12...",-57.33000,895.11993,1226.15991,1485.79004,1129.23999,1069.60010,729.96002,1131.07007,23.93000,707.69000,996.14001,669.07996
6071,Rangat,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.75932 12.16706, 92.75909 12...",-123.59998,1721.77014,2356.51978,2886.12988,2334.53003,2095.41992,769.65009,2165.02002,320.91000,1711.75000,1921.90991,1251.92993
6072,Ferrargunj,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.60467 11.53490, 92.60471 11...",306.91000,1959.29004,2564.42993,2903.42993,2319.10986,1551.25000,1472.43994,2039.08997,294.48001,1876.32007,1927.65002,1357.66003
6073,Port Blair,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.61728 10.92835, 92.61446 10...",2352.09009,3190.20020,3511.36011,3221.36011,2606.87012,2402.27002,1574.00012,2653.94019,226.66998,2216.20996,2652.66992,2502.69019


In [28]:




# Open the TIF file
with rasterio.open("/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/VIIRS_Monthly_Tiled/2020/202001.tif") as src:
    # Read the data as a NumPy array
    radiance_datacube = src.read(1)  # Assuming you are interested in the first band
    
    # Clean the data
    cleaned_datacube = replace_negative(radiance_datacube)
    
    # Extract geometry from GeoDataFrame
    geometries = gdf.geometry.values.tolist()
    
    # Mask the cleaned raster with GeoJSON geometries
    masked_data, transform = mask(src, geometries, crop=True)


In [15]:
sum_luminosity

array([12327101.], dtype=float32)

In [19]:
g2 = gpd.read_file('/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/maps-master/geojson/sub_dist.geojson')
g2

,NAME,DISTRICT,STATE_UT,geometry
0,DATA NOT AVAILABLE,DATA NOT AVAILABLE,Jammu & Kashmir,"MULTIPOLYGON (((74.77117 37.01042, 74.77968 36..."
1,NaN,Kachchh,Gujarat,"MULTIPOLYGON (((70.12473 22.97347, 70.12521 22..."
2,KACHCHH (PART),Kachchh,Gujarat,"MULTIPOLYGON (((71.30731 24.56554, 71.30731 24..."
3,Kupwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.57425 34.59931, 74.56734 34..."
4,Handwara,Kupwara,Jammu & Kashmir,"MULTIPOLYGON (((74.13457 34.44413, 74.13615 34..."
...,...,...,...,...
6070,Mayabunder,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.72334 12.73736, 92.72345 12..."
6071,Rangat,North & Middle Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.75932 12.16706, 92.75909 12..."
6072,Ferrargunj,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.60467 11.53490, 92.60471 11..."
6073,Port Blair,South Andaman,Andaman & Nicobar Islands,"MULTIPOLYGON (((92.61728 10.92835, 92.61446 10..."


In [23]:
# Read the GeoJSON file using GeoPandas
gdf = gpd.read_file("/Users/sid/Library/CloudStorage/OneDrive-DeakinUniversity/UDocs - D/DataSets/maps-master/Village/KARNATAKA.geojson")

gdf

,OBJECTID,VILNAM_SOI,DTNAME.x,STNAME,Shape_Length,Shape_Area,STCODE11,DTCODE11.x,SDTCODE11,VILCODE11,SDTNAME,VILNAME11.x,State code,State Name(In English),State census code,District code,District Name(In English),District Census code,SubDistrict code,Subdistrict Name(In English),Subdistrict census code,Village code,Village Name(In English),Block code,Block Name(In English),geometry
0,52871,CHENNAV|RANHATTI,BELGAUM,KARNATAKA,10498.62432,4022788.67506,29,555,05433,,Chikodi,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((74.64531 16.32839, 74.64533 16..."
1,53287,DONWAD,BELGAUM,KARNATAKA,13490.73166,6241188.08548,29,555,05433,597235,Chikodi,Donawad,29.00000,KARNATAKA,29.00000,527.00000,BELAGAVI,555,5433.00000,Chikodi,05433,597235.00000,Donawad,5774.00000,CHIKODI,"MULTIPOLYGON (((74.63940 16.36000, 74.64273 16..."
2,53288,BENNIHALLI,BELGAUM,KARNATAKA,6557.33370,2170500.40818,29,555,05433,597233,Chikodi,Bennihalli,29.00000,KARNATAKA,29.00000,527.00000,BELAGAVI,555,5433.00000,Chikodi,05433,597233.00000,Bennihalli,5774.00000,CHIKODI,"MULTIPOLYGON (((74.61945 16.35829, 74.61965 16..."
3,53375,BELAGALI,BELGAUM,KARNATAKA,10020.43451,5651316.23937,29,555,05433,597237,Chikodi,Belagali,29.00000,KARNATAKA,29.00000,527.00000,BELAGAVI,555,5433.00000,Chikodi,05433,597237.00000,Belagali,5774.00000,CHIKODI,"MULTIPOLYGON (((74.69108 16.35643, 74.69263 16..."
4,53377,KARGAON,BELGAUM,KARNATAKA,15647.58947,8363195.97083,29,555,05433,597234,Chikodi,Karagaon,29.00000,KARNATAKA,29.00000,527.00000,BELAGAVI,555,5433.00000,Chikodi,05433,597234.00000,Karagaon,5774.00000,CHIKODI,"MULTIPOLYGON (((74.63398 16.35603, 74.63514 16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28416,50825,PARSHENHATTI,BELGAUM,KARNATAKA,7534.63125,3553578.51755,29,555,05438,597668,Belgaum,Parsenahatti,29.00000,KARNATAKA,29.00000,527.00000,BELAGAVI,555,5438.00000,BELAGAVI,05438,597668.00000,Parsenahatti,5772.00000,BELAGAVI,"MULTIPOLYGON (((74.47549 16.05695, 74.47789 16..."
28417,50832,SUTKOTTI,BELGAUM,KARNATAKA,14943.14933,8890100.38933,29,555,05438,597782,Belgaum,Sutgatti (Ah),29.00000,KARNATAKA,29.00000,527.00000,BELAGAVI,555,5438.00000,BELAGAVI,05438,597782.00000,NAGERHAL,5772.00000,BELAGAVI,"MULTIPOLYGON (((74.51664 16.07221, 74.51711 16..."
28418,50836,MALMEMARADI,BELGAUM,KARNATAKA,15025.14625,10380345.55121,29,555,05438,,Belgaum,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((74.66236 16.01082, 74.66234 16..."
28419,50852,LINGANHATTI,BELGAUM,KARNATAKA,10031.76194,2343835.32028,29,555,05438,,Belgaum,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((74.45704 16.01912, 74.45732 16..."


In [24]:
# Convert GeoDataFrame to Plotly figure
# fig = px.choropleth_mapbox(gdf, 
#                            geojson=gdf.geometry,  # Use the geometry column from the GeoDataFrame
#                            locations=gdf.index,  # Use the index of the GeoDataFrame as locations
#                            color='DTCODE11.x',  # Specify the column you want to use for coloring
#                            color_continuous_scale="Viridis",  # Specify color scale
#                            mapbox_style="carto-positron",  # Specify the map style
#                         #    zoom=3, center={"lat": 37.0902, "lon": -95.7129},  # Specify initial zoom and center
#                            opacity=0.5,  # Specify opacity
#                         #    labels={'your_column': 'Your Label'}  # Specify labels
#                            )

# # Update layout
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# # Show the plot
# fig.show()